In [4]:
import os
import torch
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# --- CONFIGURATION ---
# IMPORTANT: Change this to the specific experiment folder you want to evaluate.
# IMAGE_DIR = "dataset_eval/exp1_sd15_2050" 
IMAGE_DIR = "dataset_eval/exp2_sd15_3050" 
# IMAGE_DIR = "dataset_eval/exp3_sd15_3050_v2" 
# IMAGE_DIR = "dataset_eval/exp4_sd15_3050_v3" 
# IMAGE_DIR = "dataset_eval/exp5_sdxl_a6000" 
OUTPUT_CSV = "aesthetic_scores.csv"
MODEL_ID = "openai/clip-vit-large-patch14"

def load_aesthetic_model(device):
    """Loads the pre-trained aesthetic scoring model."""
    print(f"Loading aesthetic model: {MODEL_ID}...")
    model = CLIPModel.from_pretrained(MODEL_ID).to(device)
    processor = CLIPProcessor.from_pretrained(MODEL_ID)
    print("Aesthetic model loaded.")
    return model, processor

def calculate_aesthetic_score(image, model, processor, device):
    """Calculates the aesthetic score for a single image."""
    inputs = processor(images=image, return_tensors="pt", text="aesthetic")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        # The logits_per_image gives a score of how well the image matches "aesthetic"
        score = outputs.logits_per_image.item()
    return score

def main():
    """Main function to process all images in a directory and save scores."""
    if not os.path.exists(IMAGE_DIR):
        print(f"Error: The directory '{IMAGE_DIR}' does not exist. Please check the path.")
        return

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    model, processor = load_aesthetic_model(device)
    
    results = []
    image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"No images found in '{IMAGE_DIR}'.")
        return

    print(f"Found {len(image_files)} images. Calculating aesthetic scores...")
    for filename in tqdm(image_files, desc="Processing Images"):
        try:
            image_path = os.path.join(IMAGE_DIR, filename)
            image = Image.open(image_path).convert("RGB")
            score = calculate_aesthetic_score(image, model, processor, device)
            results.append({"filename": filename, "aesthetic_score": score})
        except Exception as e:
            print(f"Could not process {filename}: {e}")

    if not results:
        print("No images were successfully scored.")
        return

    # Create and save a DataFrame
    df = pd.DataFrame(results)
    df.sort_values(by="aesthetic_score", ascending=False, inplace=True)
    df.to_csv(OUTPUT_CSV, index=False)
    
    average_score = df['aesthetic_score'].mean()
    print("\n--- Evaluation Complete ---")
    print(f"Results saved to {OUTPUT_CSV}")
    print(f"Average Aesthetic Score: {average_score:.4f}")
    print("\nTop 5 images with the highest aesthetic scores:")
    print(df.head(5))

if __name__ == "__main__":
    main()


Using device: cuda
Loading aesthetic model: openai/clip-vit-large-patch14...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Aesthetic model loaded.
Found 60 images. Calculating aesthetic scores...


Processing Images: 100%|██████████| 60/60 [00:06<00:00,  9.52it/s]


--- Evaluation Complete ---
Results saved to aesthetic_scores.csv
Average Aesthetic Score: 14.4154

Top 5 images with the highest aesthetic scores:
             filename  aesthetic_score
23  Gluduk_00004_.png        18.234726
24  Gluduk_00005_.png        18.217476
20  Gluduk_00001_.png        17.954725
29  Gluduk_00010_.png        17.459297
19    Dutt_00010_.png        17.136108
